# Exploración de Datos

In [3]:
import sys
import pathlib

# Agrega la raíz del proyecto a sys.path
sys.path.append(str(pathlib.Path().resolve().parent))


In [16]:
import pandas as pd
df = pd.read_parquet("C:/Users/felip/market-ia-trading-bot/data/processed/AAPL.parquet", engine="fastparquet")
print(df[["net_buy","buy_count","sell_count","amount_net"]].tail(10))


            net_buy  buy_count  sell_count  amount_net
Date                                                  
2020-03-19      0.0        0.0         0.0         NaN
2020-03-20      1.0        1.0         0.0     15001.0
2020-03-23      0.0        0.0         0.0         NaN
2020-03-24      1.0        1.0         0.0     15001.0
2020-03-25      0.0        0.0         0.0         NaN
2020-03-26      0.0        0.0         0.0         NaN
2020-03-27      0.0        0.0         0.0         NaN
2020-03-30      0.0        0.0         0.0         NaN
2020-03-31      0.0        0.0         0.0         NaN
2020-04-01      0.0        0.0         0.0         NaN


In [9]:
import os
print("Working dir:", os.getcwd())


Working dir: c:\Users\felip\market-ia-trading-bot\notebooks


In [12]:
df.columns.tolist()


['SimFinId',
 'Currency_x',
 'Fiscal Year',
 'Fiscal Period_x',
 'Publish Date_x',
 'Restated Date_x',
 'Shares (Basic)_x',
 'Shares (Diluted)_x',
 'Revenue',
 'Cost of Revenue',
 'Gross Profit',
 'Operating Expenses',
 'Selling, General & Administrative',
 'Research & Development',
 'Depreciation & Amortization_x',
 'Operating Income (Loss)',
 'Non-Operating Income (Loss)',
 'Interest Expense, Net',
 'Pretax Income (Loss), Adj.',
 'Abnormal Gains (Losses)',
 'Pretax Income (Loss)',
 'Income Tax (Expense) Benefit, Net',
 'Income (Loss) from Continuing Operations',
 'Net Extraordinary Gains (Losses)',
 'Net Income',
 'Net Income (Common)',
 'Currency_y',
 'Fiscal Period_y',
 'Publish Date_y',
 'Restated Date_y',
 'Shares (Basic)_y',
 'Shares (Diluted)_y',
 'Cash, Cash Equivalents & Short Term Investments',
 'Accounts & Notes Receivable',
 'Inventories',
 'Total Current Assets',
 'Property, Plant & Equipment, Net',
 'Long Term Investments & Receivables',
 'Other Long Term Assets',
 'Tota

In [1]:
import pandas as pd
df = pd.read_parquet("C:/Users/felip/market-ia-trading-bot/data/processed/AAPL.parquet", engine="fastparquet")
print(df.columns[:20])                    # mira qué columnas hay
print(df[["daily_return","net_buy","sentiment_score"]].tail(5))
print(df.index.min(), df.index.max(), len(df))


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'volume_adi',
       'volume_obv', 'volume_cmf', 'volume_fi', 'volume_em', 'volume_sma_em',
       'volume_vpt', 'volume_vwap', 'volume_mfi', 'volume_nvi',
       'volatility_bbm', 'volatility_bbh', 'volatility_bbl', 'volatility_bbw'],
      dtype='object')
            daily_return  net_buy  sentiment_score
Date                                              
2020-03-26      0.052623      0.0              0.0
2020-03-27     -0.041402      0.0              0.0
2020-03-30      0.028538      0.0              0.0
2020-03-31     -0.002041      0.0              0.0
2020-04-01     -0.052617      0.0              0.0
1980-12-12 00:00:00 2020-04-01 00:00:00 9909


In [3]:
from pathlib import Path
from typing import List, Tuple, Dict, Literal, Optional

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from lightgbm import LGBMRanker
from sklearn.metrics import ndcg_score
import joblib


In [5]:
from pathlib import Path

def find_repo_root(start: Path = Path.cwd()) -> Path:
    for p in [start, *start.parents]:
        if (p / "data" / "ml").exists():
            return p
    raise FileNotFoundError("No pude encontrar la carpeta data/ml subiendo por los padres.")

ROOT = find_repo_root()                 # ← auto-detecta la raíz del repo
DATA_ML = ROOT / "data" / "ml"
ARTS = ROOT / "artifacts" / "rank"
ARTS.mkdir(parents=True, exist_ok=True)

H = 36
SPLIT = "2017-01-01"
REL_BINS = 4
EVAL_K = [5, 10]
SIGN_FROM = "val"
VAL_MONTHS = 24
SIGN_THRESHOLD = 0.01
SCORE_FUTURE = True

dataset_fp = DATA_ML / f"dataset_rank_{H}m.parquet"
print("Usando DATA_ML:", DATA_ML)
assert dataset_fp.exists(), f"No existe {dataset_fp}"


Usando DATA_ML: c:\Users\felip\market-ia-trading-bot\data\ml


In [6]:
def month_key(s: pd.Series) -> pd.Series:
    return pd.to_datetime(s).dt.to_period("M").astype(str)

def make_rel_labels_per_month(y: pd.Series, month: pd.Series, bins: int) -> np.ndarray:
    if bins < 2:
        raise ValueError("rel_bins debe ser >= 2")
    df = pd.DataFrame({"y": y.values, "m": month.values})
    labels = np.full(len(df), np.nan, dtype=float)
    for m, g in df.groupby("m", sort=False):
        vals = g["y"].to_numpy(dtype=float)
        if np.isnan(vals).all():
            continue
        valid = ~np.isnan(vals)
        if valid.sum() == 0:
            continue
        ranks = np.argsort(np.argsort(vals[valid]))  # 0..n-1 ascendente
        frac = (ranks + 1) / (valid.sum() + 1e-9)
        lab = np.minimum(bins - 1, (frac * bins).astype(int))
        tmp = np.full_like(vals, np.nan, dtype=float)
        tmp[valid] = lab
        labels[g.index.values] = tmp
    out = labels
    out = np.where(np.isnan(out), -1, out)  # no debería usarse en fit si -1
    return out.astype(int)

def groups_by_month(m: pd.Series) -> np.ndarray:
    sizes = m.value_counts().sort_index()
    return sizes.to_numpy(dtype=int)

def monthly_ic(df: pd.DataFrame, score_col: str, y_col: str) -> Tuple[float, float, int, pd.DataFrame]:
    det = (
        df.groupby("month", as_index=False)
          .apply(lambda g: pd.Series({"ic": g[score_col].corr(g[y_col], method="spearman")}))
          .reset_index(drop=True)
    )
    det["ic"] = det["ic"].astype(float)
    det = det.replace([np.inf, -np.inf], np.nan).dropna(subset=["ic"])
    mean_ic = float(det["ic"].mean()) if not det.empty else np.nan
    std_ic = float(det["ic"].std(ddof=0)) if len(det) > 1 else 0.0
    return mean_ic, std_ic, int(len(det)), det

def ndcg_by_month(df: pd.DataFrame, score_col: str, y_col: str, ks: List[int]) -> Dict[int, float]:
    out: Dict[int, List[float]] = {k: [] for k in ks}
    for m, g in df.groupby("month"):
        y_true = g[y_col].to_numpy(dtype=float).reshape(1, -1)
        y_score = g[score_col].to_numpy(dtype=float).reshape(1, -1)
        if np.isnan(y_true).any() or np.isnan(y_score).any():
            continue
        for k in ks:
            k_eff = max(1, min(k, y_true.shape[1]))
            out[k].append(float(ndcg_score(y_true, y_score, k=k_eff)))
    return {k: (np.mean(v) if v else np.nan) for k, v in out.items()}

def top_bottom_spread(df: pd.DataFrame, score_col: str, y_col: str, k: int) -> Tuple[float, float]:
    spreads: List[float] = []
    pos = 0
    for m, g in df.groupby("month"):
        gg = g.sort_values(score_col, ascending=False)
        k_eff = max(1, min(k, len(gg)))
        top_y = float(gg.head(k_eff)[y_col].mean())
        bot_y = float(gg.tail(k_eff)[y_col].mean())
        sp = top_y - bot_y
        spreads.append(sp)
        if sp > 0:
            pos += 1
    if not spreads:
        return np.nan, np.nan
    return float(np.mean(spreads)), float(pos / len(spreads))


In [7]:
def month_key(s: pd.Series) -> pd.Series:
    return pd.to_datetime(s).dt.to_period("M").astype(str)

def make_rel_labels_per_month(y: pd.Series, month: pd.Series, bins: int) -> np.ndarray:
    if bins < 2:
        raise ValueError("rel_bins debe ser >= 2")
    df = pd.DataFrame({"y": y.values, "m": month.values})
    labels = np.full(len(df), np.nan, dtype=float)
    for m, g in df.groupby("m", sort=False):
        vals = g["y"].to_numpy(dtype=float)
        if np.isnan(vals).all():
            continue
        valid = ~np.isnan(vals)
        if valid.sum() == 0:
            continue
        ranks = np.argsort(np.argsort(vals[valid]))  # 0..n-1 ascendente
        frac = (ranks + 1) / (valid.sum() + 1e-9)
        lab = np.minimum(bins - 1, (frac * bins).astype(int))
        tmp = np.full_like(vals, np.nan, dtype=float)
        tmp[valid] = lab
        labels[g.index.values] = tmp
    out = labels
    out = np.where(np.isnan(out), -1, out)  # no debería usarse en fit si -1
    return out.astype(int)

def groups_by_month(m: pd.Series) -> np.ndarray:
    sizes = m.value_counts().sort_index()
    return sizes.to_numpy(dtype=int)

def monthly_ic(df: pd.DataFrame, score_col: str, y_col: str) -> Tuple[float, float, int, pd.DataFrame]:
    det = (
        df.groupby("month", as_index=False)
          .apply(lambda g: pd.Series({"ic": g[score_col].corr(g[y_col], method="spearman")}))
          .reset_index(drop=True)
    )
    det["ic"] = det["ic"].astype(float)
    det = det.replace([np.inf, -np.inf], np.nan).dropna(subset=["ic"])
    mean_ic = float(det["ic"].mean()) if not det.empty else np.nan
    std_ic = float(det["ic"].std(ddof=0)) if len(det) > 1 else 0.0
    return mean_ic, std_ic, int(len(det)), det

def ndcg_by_month(df: pd.DataFrame, score_col: str, y_col: str, ks: List[int]) -> Dict[int, float]:
    out: Dict[int, List[float]] = {k: [] for k in ks}
    for m, g in df.groupby("month"):
        y_true = g[y_col].to_numpy(dtype=float).reshape(1, -1)
        y_score = g[score_col].to_numpy(dtype=float).reshape(1, -1)
        if np.isnan(y_true).any() or np.isnan(y_score).any():
            continue
        for k in ks:
            k_eff = max(1, min(k, y_true.shape[1]))
            out[k].append(float(ndcg_score(y_true, y_score, k=k_eff)))
    return {k: (np.mean(v) if v else np.nan) for k, v in out.items()}

def top_bottom_spread(df: pd.DataFrame, score_col: str, y_col: str, k: int) -> Tuple[float, float]:
    spreads: List[float] = []
    pos = 0
    for m, g in df.groupby("month"):
        gg = g.sort_values(score_col, ascending=False)
        k_eff = max(1, min(k, len(gg)))
        top_y = float(gg.head(k_eff)[y_col].mean())
        bot_y = float(gg.tail(k_eff)[y_col].mean())
        sp = top_y - bot_y
        spreads.append(sp)
        if sp > 0:
            pos += 1
    if not spreads:
        return np.nan, np.nan
    return float(np.mean(spreads)), float(pos / len(spreads))


In [12]:
y_col = f"y_fwd_{H}m"
df = pd.read_parquet(dataset_fp)
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df["month"] = pd.to_datetime(df["Date"]).dt.to_period("M").astype(str)

# split
df["split"] = "train"
df.loc[(df["Date"] >= pd.to_datetime(SPLIT)) & (df[y_col].notna()), "split"] = "test"
df.loc[(df["Date"] >= pd.to_datetime(SPLIT)) & (df[y_col].isna()), "split"] = "future"

tr = df[(df["split"] == "train") & df[y_col].notna()].copy()
te = df[(df["split"] == "test") & df[y_col].notna()].copy()
fu = df[(df["split"] == "future")].copy()

# muy importante: ordenar por month para que 'group' coincida con el orden de las filas
tr = tr.sort_values(["month", "Ticker"]).reset_index(drop=True)
te = te.sort_values(["month", "Ticker"]).reset_index(drop=True)
fu = fu.sort_values(["month", "Ticker"]).reset_index(drop=True)

print(f"train={len(tr):,} | test={len(te):,} | future={len(fu):,}")
print("Rangos:",
      f"train {tr['Date'].min().date()}..{tr['Date'].max().date()} |",
      f"test {te['Date'].min().date() if not te.empty else 'NA'}..{te['Date'].max().date() if not te.empty else 'NA'}")


train=292,704 | test=8,948 | future=84,631
Rangos: train 1973-05-31..2016-12-31 | test 2017-01-31..2017-04-30


In [13]:
drop_cols = {"Date", "Ticker", y_col, "split", "month"}
numeric_cols = [c for c in df.columns if c not in drop_cols and pd.api.types.is_numeric_dtype(df[c])]
categorical_cols = [c for c in ["Ticker"] if c in df.columns]

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

pre = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=True), categorical_cols),
        ("num", "passthrough", numeric_cols),
    ],
    remainder="drop",
    sparse_threshold=1.0,
)


In [ ]:
def make_rel_labels_per_month(y: pd.Series, month: pd.Series, bins: int) -> np.ndarray:
    if bins < 2:
        raise ValueError("rel_bins debe ser >= 2")
    df_ = pd.DataFrame({"y": y.values, "m": month.values})
    out = np.full(len(df_), np.nan, dtype=float)
    for m, g in df_.groupby("m", sort=False):
        vals = g["y"].to_numpy(dtype=float)
        valid = ~np.isnan(vals)
        if valid.sum() == 0:
            continue
        ranks = np.argsort(np.argsort(vals[valid]))
        frac = (ranks + 1) / (valid.sum() + 1e-9)
        lab = np.minimum(REL_BINS - 1, (frac * REL_BINS).astype(int))
        tmp = np.full_like(vals, np.nan, dtype=float)
        tmp[valid] = lab
        out[g.index.values] = tmp
    out = np.where(np.isnan(out), -1, out)
    return out.astype(int)

def groups_from_sorted_month(series_month: pd.Series) -> np.ndarray:
    # asume que el DataFrame YA está ordenado por 'month'
    return series_month.value_counts(sort=False).reindex(series_month.drop_duplicates(), fill_value=0).to_numpy()

tr["rel_label"] = make_rel_labels_per_month(tr[y_col], tr["month"], REL_BINS)
te["rel_label"] = make_rel_labels_per_month(te[y_col], te["month"], REL_BINS)

g_tr = groups_from_sorted_month(tr["month"])
g_te = groups_from_sorted_month(te["month"]) if not te.empty else None

# chequeos
assert g_tr.sum() == len(tr), "group train no cuadra con filas"
if g_te is not None:
    assert g_te.sum() == len(te), "group test no cuadra con filas"


In [15]:
from lightgbm import LGBMRanker

# ganancias por bin (peso a la parte alta)
gains = [0] + [int(round(2 ** (i - 1))) for i in range(1, REL_BINS)]

model = LGBMRanker(
    objective="lambdarank",
    random_state=42,
    n_estimators=600,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.8,
    metric="ndcg",
    eval_at=[int(k) for k in EVAL_K],
    label_gain=gains,
    n_jobs=-1,
    # regularización más fuerte si quieres:
    # num_leaves=31, min_data_in_leaf=100, reg_lambda=1.0, reg_alpha=0.0
)

# 1) fit del preprocessor en TRAIN
Xtr = tr[numeric_cols + categorical_cols]
pre.fit(Xtr)

# 2) transformar TRAIN y (si existe) TEST
Xtr_t = pre.transform(Xtr)
ytr = tr["rel_label"].astype(int).values

eval_sets = None
eval_groups = None
if not te.empty:
    Xte = te[numeric_cols + categorical_cols]
    Xte_t = pre.transform(Xte)     # <--- transformado
    yte = te["rel_label"].astype(int).values
    eval_sets = [(Xte_t, yte)]
    eval_groups = [g_te]

# 3) entrenar el ranker con grupos
if eval_sets is not None:
    model.fit(Xtr_t, ytr, group=g_tr, eval_set=eval_sets, eval_group=eval_groups)
else:
    model.fit(Xtr_t, ytr, group=g_tr)

# 4) construir un Pipeline "ya entrenado"
from sklearn.pipeline import Pipeline
pipe = Pipeline([("prep", pre), ("model", model)])


c:\Users\felip\anaconda3\envs\market-ia-trading-bot\Lib\site-packages\lightgbm\sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.066452 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8986
[LightGBM] [Info] Number of data points in the train set: 292704, number of used features: 2469


In [16]:
# predicciones
tr["raw_score"] = pipe.predict(tr[numeric_cols + categorical_cols]).astype(float)
te["raw_score"] = pipe.predict(te[numeric_cols + categorical_cols]).astype(float)
if SCORE_FUTURE and not fu.empty:
    fu["raw_score"] = pipe.predict(fu[numeric_cols + categorical_cols]).astype(float)

# ... luego tu celda de chequeo de signo (IC) y flip ...


c:\Users\felip\anaconda3\envs\market-ia-trading-bot\Lib\site-packages\lightgbm\sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
c:\Users\felip\anaconda3\envs\market-ia-trading-bot\Lib\site-packages\lightgbm\sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
c:\Users\felip\anaconda3\envs\market-ia-trading-bot\Lib\site-packages\lightgbm\sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


In [17]:
if SIGN_FROM == "val":
    months_tr = sorted(tr["month"].unique())
    val_set = set(months_tr[-max(1, VAL_MONTHS):])
    chk = tr[tr["month"].isin(val_set)].copy()
else:
    chk = te.copy()

direction = +1
ic_mean = np.nan
ic_std = np.nan
ic_n = 0
if not chk.empty:
    chk["score_tmp"] = chk["raw_score"]
    ic_mean, ic_std, ic_n, det = monthly_ic(chk, "score_tmp", y_col)
    print(f"[SIGN CHECK={SIGN_FROM}] IC mean={ic_mean:.4f}, std={ic_std:.4f}, n_months={ic_n}")
    if np.isfinite(ic_mean) and ic_mean < -abs(float(SIGN_THRESHOLD)):
        direction = -1
        print("→ Señal invertida (flip) por IC medio negativo.")

for fr in (tr, te, fu):
    if not fr.empty:
        fr["score"] = direction * fr["raw_score"]


[SIGN CHECK=val] IC mean=0.1876, std=0.0215, n_months=24


C:\Users\felip\AppData\Local\Temp\ipykernel_3904\4159941757.py:33: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({"ic": g[score_col].corr(g[y_col], method="spearman")}))


In [18]:
if not te.empty:
    ic_mean_te, ic_std_te, ic_n_te, _ = monthly_ic(te, "score", y_col)
    ndcg_te = ndcg_by_month(te, "score", y_col, EVAL_K)
    print(f"IC mensual (test) — mean={ic_mean_te:.4f}, std={ic_std_te:.4f}, n={ic_n_te}")
    print("NDCG@K (test):", {k: float(v) for k,v in ndcg_te.items()})

    for k in EVAL_K:
        sp, pos = top_bottom_spread(te, "score", y_col, k)
        print(f"Spread Top{k}-Bottom{k} — mean={sp:.4f}, hit%={pos:.1%}")


IC mensual (test) — mean=0.0542, std=0.0600, n=4
NDCG@K (test): {5: 0.11668324787776539, 10: 0.12663317324293194}
Spread Top5-Bottom5 — mean=1.4657, hit%=100.0%
Spread Top10-Bottom10 — mean=1.1101, hit%=100.0%


C:\Users\felip\AppData\Local\Temp\ipykernel_3904\4159941757.py:33: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({"ic": g[score_col].corr(g[y_col], method="spearman")}))
c:\Users\felip\anaconda3\envs\market-ia-trading-bot\Lib\site-packages\sklearn\metrics\_ranking.py:1807: FutureWarning: ndcg_score should not be used on negative y_true values. ndcg_score will raise a ValueError on negative y_true values starting from version 1.4.
  warnings.warn(
c:\Users\felip\anaconda3\envs\market-ia-trading-bot\Lib\site-packages\sklearn\metrics\_ranking.py:1807: FutureWarning: ndcg_score should not be used on negative y_true values. ndcg_score will raise a ValueError on negative y_true va

In [19]:
# Modelo
model_fp = ARTS / f"ranker_{H}m_lgbm.pkl"
joblib.dump(pipe, model_fp)
print("✅ Modelo guardado en:", model_fp)

# Predicciones
preds = pd.concat([
    tr[["Date", "Ticker", "split", "raw_score", "score"]],
    te[["Date", "Ticker", "split", "raw_score", "score"]],
    fu[["Date", "Ticker", "split", "raw_score", "score"]],
], ignore_index=True).sort_values(["Date","Ticker"])

out_fp = DATA_ML / f"preds_rank_{H}m.parquet"
preds.to_parquet(out_fp, index=False)
print("✅ Predicciones guardadas en:", out_fp, "| filas:", len(preds))


✅ Modelo guardado en: c:\Users\felip\market-ia-trading-bot\artifacts\rank\ranker_36m_lgbm.pkl
✅ Predicciones guardadas en: c:\Users\felip\market-ia-trading-bot\data\ml\preds_rank_36m.parquet | filas: 386283
